In [1]:
import re
import pandas as pd
import numpy as np
# from mlutils import dataset, connector
import scipy.stats
from scipy.stats import *
import scipy.stats as stats

from sklearn.preprocessing import StandardScaler
import math
import matplotlib.pyplot as plt
import warnings
import statsmodels.api as sm 
import seaborn as sns
import pylab as py 
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
def standarise(y,pct,pct_lower):
    sc = StandardScaler() 
    y.sort()
    len_y = len(y)
    y = y[int(pct_lower * len_y):int(len_y * pct)]
    len_y = len(y)
    yy=([[x] for x in y])
    sc.fit(yy)
    y_std =sc.transform(yy)
    y_std = y_std.flatten()
    return y_std,len_y,y

In [3]:
def fit_distribution(column,pct,pct_lower):
    # Set up list of candidate distributions to use
    # See https://docs.scipy.org/doc/scipy/reference/stats.html for more
    y_std,size,y_org = standarise(column,pct,pct_lower)
    dist_names = ['weibull_min','norm','weibull_max','beta',
                 'invgauss','uniform','gamma','expon', 'lognorm','pearson3','triang','erlang']

    chi_square_statistics = []
    # 11 bins
    percentile_bins = np.linspace(0,100,11)
    percentile_cutoffs = np.percentile(y_std, percentile_bins)
    observed_frequency, bins = (np.histogram(y_std, bins=percentile_cutoffs))
    cum_observed_frequency = np.cumsum(observed_frequency)

    # Loop through candidate distributions

    for distribution in dist_names:
        # Set up distribution and get fitted distribution parameters
        dist = getattr(scipy.stats, distribution)
        param = dist.fit(y_std)
        print("{}\n{}\n".format(dist, param))
      

        # Get expected counts in percentile bins
        # cdf of fitted sistrinution across bins
        cdf_fitted = dist.cdf(percentile_cutoffs, *param)
        expected_frequency = []
        for bin in range(len(percentile_bins)-1):
            expected_cdf_area = cdf_fitted[bin+1] - cdf_fitted[bin]
            expected_frequency.append(expected_cdf_area)

        # Chi-square Statistics
        expected_frequency = np.array(expected_frequency) * size
        cum_expected_frequency = np.cumsum(expected_frequency)
        ss = round(sum (((cum_expected_frequency - cum_observed_frequency) ** 2) / cum_observed_frequency),6)
        chi_square_statistics.append(ss)


    #Sort by minimum ch-square statistics
    results = pd.DataFrame()
    results['Distribution'] = dist_names
    results['chi_square'] = chi_square_statistics
    results.sort_values(['chi_square'], inplace=True)


    print('\nDistributions listed by Betterment of fit:')
    print('............................................')
    print(results)

## Interarrival times and Service times

In [4]:
import read_data as rd

In [5]:
import os
from pathlib import Path

filenames = [str(Path(os.path.abspath('')).parent) + "\Data Collection\KevinYeData.txt",
            str(Path(os.path.abspath('')).parent) + "\Data Collection\dataviviandong.txt",
            str(Path(os.path.abspath('')).parent) + "\Data Collection\PatrickQData.txt",
            str(Path(os.path.abspath('')).parent) + "\Data Collection\TamaHoareData.txt"]
results = rd.ReadData(filenames=filenames)
AllArrivals = results[0]
AllServices = results[1]

In [6]:
fit_distribution(AllArrivals,0.99,0.01)

(0.7015384049768638, -0.9894365309023049, 1.2006676444778561)

(0.0, 1.0)

(0.7019179793807708, 3.26754168282175, 1.2352709611338757)

(0.5869141736933967, 2.0230692974358018, -0.9894365309023049, 5.611707278987666)

(1.3774125862615234, -1.133748597072893, 0.8230934376760495)

(-0.9894365309023048, 4.256978213724054)

(0.8667586476420364, -0.989436530902305, 0.8160179858473148)

(-0.9894365309023048, 0.9894365309023048)

(1.2285098631304354, -1.040346141653061, 0.5785087400178923)

(2.416641101443037, -0.007126039833642914, 1.1869459535476108)

(1.2422294892673428e-07, -0.9894371622132272, 4.384475250138744)

(0.34939810853283576, -0.9894365309023049, 1.0870659237131632)


Distributions listed by Betterment of fit:
............................................
   Distribution   chi_square
9      pearson3     9.155252
0   weibull_min    13.245287
3          beta    21.708357
8       lognorm    25.596288
4      invgauss    29.389634
7         expon    29.515278
6         gamma    48.3593

In [7]:
fit_distribution(AllServices,0.99,0.01)

(1.2771714921950053, -1.2958750507447454, 1.3965158029229956)

(-1.3448758405553947e-16, 0.9999999999999999)

(0.7088599016352515, 3.1486712840616633, 1.2507073667198347)

(1.169328787311814, 4.603218177027601, -1.2938279407948883, 6.418076728180601)

(0.4698497549488414, -1.6286272546397809, 3.4662638467893183)

(-1.2924111429457916, 4.441082427007455)

(1.4735014952989527, -1.3003337972240427, 0.882459117699409)

(-1.2924111429457916, 1.2924111429457914)

(0.6731930530194032, -1.5554556326439215, 1.258664637941075)

(1.6476143121145541, -1.5950207493546993e-17, 1.0712335638157982)

(0.02332622632016261, -1.3149179115755778, 4.59688627626827)

(1.4735836332147951, -1.300343744879871, 0.8824094844571524)


Distributions listed by Betterment of fit:
............................................
   Distribution   chi_square
3          beta     1.231338
0   weibull_min     2.831316
9      pearson3     4.130412
6         gamma     4.131762
11       erlang     4.132443
4      invgauss    10.

## Fitting different Distributions and checking Goodness of fit based on Chi-square Statistics.

Based on the Chi-sqaure Statistics, it suggests that Pearson3 is the best in approximating ‘inter-arrival time’ Data.

We can also see that beta distribution is the best fit for service times.

The code used in this file is from https://github.com/mungoliabhishek/Distribution-Fitting-Used_Car_Dataset/blob/master/Workbook.ipynb